In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [16]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema('microns_ta3p100')


In [7]:
#ta3p100.ProofreadLabelFinal & 'status="complete"'

In [12]:
@schema
class CoarseLabelFinal(dj.Computed):
    definition = """
    # Vertex labels for ta3p100.ProofreadLabel did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> ta3p100.ProofreadLabelFinal
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as ta3p100.ProofreadLabel
    """
    
    key_source = ta3p100.ProofreadLabelFinal & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (ta3p100.ProofreadLabelFinal & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (ta3p100.CleansedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [13]:
start = time.time()
CoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346341366885
Segment 648518346341366885 vertex labels regenerated in: 23.73721957206726 seconds.
648518346341371119
Segment 648518346341371119 vertex labels regenerated in: 2.68520188331604 seconds.
648518346341388820
Segment 648518346341388820 vertex labels regenerated in: 2.3023300170898438 seconds.
648518346341393609
Segment 648518346341393609 vertex labels regenerated in: 3.1707770824432373 seconds.
648518346342797556
Segment 648518346342797556 vertex labels regenerated in: 2.834840774536133 seconds.
648518346342806978
Segment 648518346342806978 vertex labels regenerated in: 2.773287773132324 seconds.
648518346349386137
Segment 648518346349386137 vertex labels regenerated in: 1.8811750411987305 seconds.
648518346349468264
Segment 648518346349468264 vertex labels regenerated in: 2.785353660583496 seconds.
648518346349470171
Segment 648518346349470171 vertex labels regenerated in: 2.617609739303589 seconds.
648518346349471156
Segment 648518346349471156 vertex labels regenerated

In [11]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()

In [17]:
ta3p100.CoarseLabelFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as ta3p100.ProofreadLabel
2,648518346341366885,0.35,computer_Auto,bcelii,2019-02-01 12:28:16,=BLOB=,=BLOB=
2,648518346341371119,0.35,computer_Auto,bcelii,2019-02-01 12:35:29,=BLOB=,=BLOB=
2,648518346341388820,0.35,computer_Auto,bcelii,2019-02-01 12:36:10,=BLOB=,=BLOB=
2,648518346341393609,0.35,computer_Auto,bcelii,2019-02-01 12:39:11,=BLOB=,=BLOB=
2,648518346342797556,0.35,computer_Auto,bcelii,2019-02-01 12:41:08,=BLOB=,=BLOB=
2,648518346342806978,0.35,computer_Auto,bcelii,2019-02-01 13:05:20,=BLOB=,=BLOB=
2,648518346349386137,0.35,computer_Auto,bcelii,2019-02-01 13:05:47,=BLOB=,=BLOB=
2,648518346349468264,0.35,computer_Auto,bcelii,2019-02-01 13:06:38,=BLOB=,=BLOB=
2,648518346349470171,0.35,computer_Auto,bcelii,2019-02-01 13:08:02,=BLOB=,=BLOB=
2,648518346349471156,0.35,computer_Auto,bcelii,2019-02-01 13:36:25,=BLOB=,=BLOB=
